In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 

In [4]:
class DataLoader():
    def __init__(self):
        path = tf.keras.utils.get_file('nietzsche.txt',
            origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
        with open(path, encoding='utf-8') as f:
            self.raw_text = f.read().lower()
        self.chars = sorted(list(set(self.raw_text)))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        self.text = [self.char_indices[c] for c in self.raw_text]

    def get_batch(self, seq_length, batch_size):
        seq = []
        next_char = []
        for i in range(batch_size):
            index = np.random.randint(0, len(self.text) - seq_length)
            seq.append(self.text[index:index+seq_length])
            next_char.append(self.text[index+seq_length])
        return np.array(seq), np.array(next_char)       # [batch_size, seq_length], [num_batch]

In [5]:
DataLoader()

In [6]:
class RNN(tf.keras.Model):
    def __init__(self, num_chars, batch_size, seq_length):
        super().__init__()
        self.num_chars = num_chars
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.cell = tf.keras.layers.LSTMCell(units=256)
        self.dense = tf.keras.layers.Dense(units=self.num_chars)

    def call(self, inputs, from_logits=False):
        inputs = tf.one_hot(inputs, depth=self.num_chars)       # [batch_size, seq_length, num_chars]
        state = self.cell.get_initial_state(batch_size=self.batch_size, dtype=tf.float32)
        for t in range(self.seq_length):
            output, state = self.cell(inputs[:, t, :], state)
        logits = self.dense(output)
        if from_logits:
            return logits
        else:
            return tf.nn.softmax(logits)

In [7]:
num_batches = 1000
seq_length = 40
batch_size = 50
learning_rate = 1e-3

In [8]:
data_loader = DataLoader()
model = RNN(num_chars=len(data_loader.chars), batch_size=batch_size, seq_length=seq_length)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(seq_length, batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 4.038029
batch 1: loss 4.018785
batch 2: loss 3.994541
batch 3: loss 3.967263
batch 4: loss 3.877993
batch 5: loss 3.786879
batch 6: loss 3.376133
batch 7: loss 3.591201
batch 8: loss 3.229809
batch 9: loss 3.399043
batch 10: loss 3.204137
batch 11: loss 3.046906
batch 12: loss 3.152682
batch 13: loss 3.080732
batch 14: loss 2.969451
batch 15: loss 3.026066
batch 16: loss 3.051837
batch 17: loss 3.345060
batch 18: loss 2.978452
batch 19: loss 3.398625
batch 20: loss 3.050918
batch 21: loss 3.316294
batch 22: loss 3.015188
batch 23: loss 2.935402
batch 24: loss 3.020400
batch 25: loss 3.273469
batch 26: loss 3.168616
batch 27: loss 3.049195
batch 28: loss 3.075008
batch 29: loss 2.872312
batch 30: loss 3.046339
batch 31: loss 3.127310
batch 32: loss 3.117383
batch 33: loss 3.055665
batch 34: loss 3.152632
batch 35: loss 3.007311
batch 36: loss 3.040401
batch 37: loss 2.981986
batch 38: loss 3.292441
batch 39: loss 3.010469
batch 40: loss 3.232718
batch 41: loss 2.947930
ba

batch 333: loss 3.108130
batch 334: loss 2.744048
batch 335: loss 2.788225
batch 336: loss 2.667076
batch 337: loss 2.925707
batch 338: loss 2.795533
batch 339: loss 2.936799
batch 340: loss 2.808050
batch 341: loss 2.669323
batch 342: loss 2.923522
batch 343: loss 2.745440
batch 344: loss 2.651659
batch 345: loss 2.794531
batch 346: loss 2.672913
batch 347: loss 3.174884
batch 348: loss 2.731817
batch 349: loss 2.667836
batch 350: loss 2.915717
batch 351: loss 2.941373
batch 352: loss 2.621330
batch 353: loss 2.724606
batch 354: loss 2.753261
batch 355: loss 3.008926
batch 356: loss 2.801748
batch 357: loss 2.657449
batch 358: loss 2.702275
batch 359: loss 2.825028
batch 360: loss 2.665081
batch 361: loss 3.139986
batch 362: loss 2.719180
batch 363: loss 3.194463
batch 364: loss 2.541144
batch 365: loss 2.637795
batch 366: loss 2.987697
batch 367: loss 2.766572
batch 368: loss 2.837399
batch 369: loss 2.794475
batch 370: loss 2.974923
batch 371: loss 2.923979
batch 372: loss 2.836269


batch 661: loss 2.404526
batch 662: loss 2.562866
batch 663: loss 2.464996
batch 664: loss 2.479268
batch 665: loss 2.584188
batch 666: loss 2.429455
batch 667: loss 2.313926
batch 668: loss 2.544755
batch 669: loss 2.705447
batch 670: loss 2.665357
batch 671: loss 2.381393
batch 672: loss 2.478146
batch 673: loss 2.878117
batch 674: loss 2.228875
batch 675: loss 2.634471
batch 676: loss 2.373473
batch 677: loss 2.571218
batch 678: loss 2.620503
batch 679: loss 2.201135
batch 680: loss 2.619467
batch 681: loss 2.577143
batch 682: loss 2.350256
batch 683: loss 2.594701
batch 684: loss 2.252181
batch 685: loss 2.617167
batch 686: loss 2.195329
batch 687: loss 2.576474
batch 688: loss 2.602294
batch 689: loss 2.708124
batch 690: loss 2.557644
batch 691: loss 2.304030
batch 692: loss 2.543303
batch 693: loss 2.767826
batch 694: loss 2.470592
batch 695: loss 2.308557
batch 696: loss 2.323673
batch 697: loss 2.174050
batch 698: loss 2.940674
batch 699: loss 2.347711
batch 700: loss 2.230034


batch 989: loss 2.180336
batch 990: loss 2.004422
batch 991: loss 2.216562
batch 992: loss 2.503207
batch 993: loss 2.183556
batch 994: loss 1.993610
batch 995: loss 2.401442
batch 996: loss 2.321387
batch 997: loss 2.302971
batch 998: loss 2.549250
batch 999: loss 2.462742


In [9]:
def predict(self, inputs, temperature=1.):
    batch_size, _ = tf.shape(inputs)
    logits = self(inputs, from_logits=True)
    prob = tf.nn.softmax(logits / temperature).numpy()
    return np.array([np.random.choice(self.num_chars, p=prob[i, :])
                     for i in range(batch_size.numpy())])

In [11]:
X_, _ = data_loader.get_batch(seq_length, 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    X = X_
    print("diversity %f:" % diversity)
    for t in range(400):
        y_pred = model.predict(X, diversity)
        print(data_loader.indices_char[y_pred[0]], end='', flush=True)
        X = np.concatenate([X[:, 1:], np.expand_dims(y_pred, axis=1)], axis=-1)
    print("\n")

diversity 0.200000:


TypeError: slice indices must be integers or None or have an __index__ method